In [1]:
import os
import json
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, ToggleButtons, ToggleButton, Layout

import numpy as np
import pandas as pd
import qgrid

import requests
from urllib.parse import urljoin
from jinja2 import Environment, FileSystemLoader
from IPython.display import display

In [2]:
base_url = 'http://127.0.0.1:8000'

In [3]:
session = requests.Session()

In [4]:
template_loader = FileSystemLoader(searchpath="./templates")
template_env = Environment(loader=template_loader)


In [5]:
all_topics_template = template_env.get_template('all_topics.html')
topic_template = template_env.get_template('show_topic.html')
document_template = template_env.get_template('show_document.html')

In [6]:
def get_number_of_topics(session):
    r = session.get(urljoin(base_url, '/topics/number'))
    return r.json()['num_topics']

In [7]:
num_topics = get_number_of_topics(session)

In [8]:
def get_topics(session, num_topics):
    payload = {'num_topics':num_topics}
    r = session.get(urljoin(base_url, '/topics/get-topics'), params=payload)
    return r.json()

In [9]:
topics = get_topics(session, num_topics)
topics_dict = {t['topic_num']: t['topic_words'] for t in topics}

In [10]:
top_words_in_topic = [(t['topic_num'], ', '.join(t['topic_words'][:10])) for t in topics]

In [11]:
rendered_all_topics = all_topics_template.render(topics=top_words_in_topic)

In [12]:
def search_contains_word(session, word):
    payload = {'keywords': [word], 'keywords_neg': [], 'num_docs': 5}
    r = session.post(urljoin(base_url, '/documents/search-by-keyword'), data=json.dumps(payload))
    return r.json()

In [13]:
initial_topic_num = 0

In [14]:
topic_select_textbox = widgets.BoundedIntText(
                        value=initial_topic_num,
                        min=0,
                        max=num_topics-1,
                        step=1,
                        description='',
                        disabled=False,
                    )

In [15]:
topic_select_slider = widgets.IntSlider(
    value=initial_topic_num,
    min=0,
    max=num_topics-1,
    step=1,
    description='Topic:',
    disabled=False,
    orientation='horizontal',
    readout=False,
    readout_format='d',
)

In [16]:
topic_words_widget = widgets.HTML(
        value = all_topics_template.render(topics=top_words_in_topic)
    )
topic_words_formatted = HBox([topic_words_widget], layout=Layout(height='250px', overflow_y='auto'))

In [17]:
topic_num_link = widgets.link((topic_select_slider, 'value'), (topic_select_textbox, 'value'))

In [18]:
def on_show_all_buttons_check(change):
    if change.new:
        rendered_all_topics = all_topics_template.render(topics=top_words_in_topic)
        topic_words_widget.value = rendered_all_topics
        topic_words_formatted.layout = Layout(height='250px', overflow_y='auto')
    else:
        topic_words_widget.value = ''
        topic_words_formatted.layout.display = 'none'

In [19]:
show_all_topics_check = widgets.Checkbox(value=True, description='Show top words for all topics', indent=False)
show_all_topics_check.observe(on_show_all_buttons_check, 'value')

In [20]:
related_docs_widget = widgets.HTML(
        value = ''
    )

In [21]:
def on_button_group_click(change):
    if not change['owner'].options:
        related_docs_widget.value = ''
    elif (change['new'] == change['owner'].options[0]) and (change['old'] not in change['owner'].options):
        related_docs_widget.value = ''
    else:
        word = change['new']
        related_docs = search_contains_word(session, word)
        rendered_documents = document_template.render(documents=related_docs, character_limit=300)
        related_docs_widget.value = rendered_documents

In [22]:
word_search_buttons = ToggleButtons(options=[], disabled=False)
word_search_buttons.observe(on_button_group_click, 'value')

In [23]:
if initial_topic_num != num_topics:
    word_search_buttons.options = topics_dict[initial_topic_num]

In [24]:
def handle_slider_change(change):
    topic_num = change.new
    word_search_buttons.options = [] # This is hacky
    word_search_buttons.options = topics_dict[topic_num]

topic_select_slider.observe(handle_slider_change, names='value')

In [25]:
overview = widgets.HTML('<h1>Topic explorer</h1>')
individual_topic_overview = widgets.HTML(
"""<h3>Investigate a specific topic</h3>
Move the slider (or use the textbox) to see all of the words for a specific topic. You can click on a word
to see sample documents containing that word.
""")

In [26]:
top_box = HBox([topic_select_slider, topic_select_textbox])
topic_explorer = VBox([overview, show_all_topics_check, topic_words_formatted, individual_topic_overview, 
                       top_box, word_search_buttons, related_docs_widget])

In [27]:
topic_explorer

In [ ]:
# buttons = ToggleButtons(options=topics_dict[0])
# # buttons = ToggleButtons(options=[])
# buttons

In [ ]:
# def on_button_group_click(change):
#     if change['new']:
#         print(change)
# #         word = change['new']
# #         related_docs = search_contains_word(session, word)
# #         rendered_documents = document_template.render(documents=related_docs, character_limit=300)
# #         related_docs_widget.value = rendered_documents
        
        
# buttons.observe(on_button_group_click, 'value')

In [ ]:
# toggle = ToggleButton(description='tommy')
# VBox([toggle, related_docs_widget])

In [ ]:
# def on_word_button_click(change):
#     if change['new']:
#         word = change['owner'].description
#         related_docs = search_contains_word(session, word)
#         rendered_documents = document_template.render(documents=related_docs, character_limit=300)
#         related_docs_widget.value = rendered_documents
        
        
# toggle.observe(on_word_button_click, 'value')

In [ ]:
# related_docs = search_contains_word(session, 'tommy')
# related_docs

In [ ]:
# rendered_documents = document_template.render(documents=related_docs, character_limit=300)

In [ ]:
# related_docs_widget = widgets.HTML(
#         value = ''
#     )

# related_docs_widget

In [ ]:
# word = 'handguns'
# payload = {'keywords': [word], 'keywords_neg': [], 'num_docs': 5}
# r = session.post(urljoin(base_url, '/documents/search-by-keyword'), data=json.dumps(payload))
# r.status_code

In [ ]:
# r.json()

In [ ]:
# payload = {'topic_num': 5, 'num_docs': 10}
# results = session.`get(urljoin(base_url, '/documents/search-by-topic'), params=payload)

In [ ]:
# results.json()

In [ ]:
# payload = {'num_topics':77}
# results = session.get(urljoin(base_url, '/topics/get-topics'), params=payload)
# results.json()